In [1]:
import sys
import mysql.connector

def connect(user, pw):
    conn = mysql.connector.connect(host='localhost', database='coffee', user=user, password=pw)
    conn.autocommit = True
    return conn

rootconn = connect('root','')
def sqlQuery(sqlString, conn=rootconn):
    try:
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchall()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

def sqlDo(sqlString, conn=rootconn):
    try:
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchwarnings()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

In [2]:
sqlQuery("show databases")

[('information_schema',),
 ('classicmodels',),
 ('coffee',),
 ('mysql',),
 ('performance_schema',)]

In [5]:
sqlDo("""
CREATE PROCEDURE denormalizeComments(IN postId INT)
BEGIN     
    update posts SET comments = (
    select JSON_ARRAYAGG(JSON_OBJECT(
    'ID',`Id`,
    'PostID', `PostId`,
    'Score', `Score`,
    'Text', `Text`,
    'CreationDate', `CreationDate`,
    'UserId', `UserId`
    ))
    from comments where PostID = postID 
    
    )
    WHERE Id = postID;
END
""")

In [7]:
sqlQuery("call coffee.denormalizeComments('1')")

1406 (22001): Data too long for column 'comments' at row 3594
